## Load Data

In [ ]:
import polars as pl
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import accuracy_score, confusion_matrix, brier_score_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from scipy import stats

In [ ]:
!pip install kaggle

In [ ]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)
# Use the file upload widget
from google.colab import files
files.upload()  # Upload your kaggle.json file here

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"beyzabal0","key":"91cff5f1980f29e22e2d53dcc5a1d50b"}'}

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json
!cp kaggle.json /root/.kaggle/

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle competitions download -c home-credit-credit-risk-model-stability

100% 3.14G/3.14G [01:06<00:00, 48.3MB/s]
100% 3.14G/3.14G [01:06<00:00, 50.5MB/s]


In [ ]:
def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    for col in df.columns:
        if col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))

    return df

def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

In [ ]:
!unzip home-credit-credit-risk-model-stability.zip

Archive:  home-credit-credit-risk-model-stability.zip
  inflating: csv_files/test/test_applprev_1_0.csv  
  inflating: csv_files/test/test_applprev_1_1.csv  
  inflating: csv_files/test/test_applprev_1_2.csv  
  inflating: csv_files/test/test_applprev_2.csv  
  inflating: csv_files/test/test_base.csv  
  inflating: csv_files/test/test_credit_bureau_a_1_0.csv  
  inflating: csv_files/test/test_credit_bureau_a_1_1.csv  
  inflating: csv_files/test/test_credit_bureau_a_1_2.csv  
  inflating: csv_files/test/test_credit_bureau_a_1_3.csv  
  inflating: csv_files/test/test_credit_bureau_a_1_4.csv  
  inflating: csv_files/test/test_credit_bureau_a_2_0.csv  
  inflating: csv_files/test/test_credit_bureau_a_2_1.csv  
  inflating: csv_files/test/test_credit_bureau_a_2_10.csv  
  inflating: csv_files/test/test_credit_bureau_a_2_11.csv  
  inflating: csv_files/test/test_credit_bureau_a_2_2.csv  
  inflating: csv_files/test/test_credit_bureau_a_2_3.csv  
  inflating: csv_files/test/test_credit_burea

In [ ]:
dataPath = "/content/"

In [ ]:
train_basetable = pl.read_csv(dataPath + "csv_files/train/train_base.csv")
train_static = pl.concat(
    [
        pl.read_csv(dataPath + "csv_files/train/train_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/train/train_static_0_1.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
train_static_cb = pl.read_csv(dataPath + "csv_files/train/train_static_cb_0.csv").pipe(set_table_dtypes)
train_person_1 = pl.read_csv(dataPath + "csv_files/train/train_person_1.csv").pipe(set_table_dtypes)
train_credit_bureau_b_2 = pl.read_csv(dataPath + "csv_files/train/train_credit_bureau_b_2.csv").pipe(set_table_dtypes)

In [ ]:
test_basetable = pl.read_csv(dataPath + "csv_files/test/test_base.csv")
test_static = pl.concat(
    [
        pl.read_csv(dataPath + "csv_files/test/test_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/test/test_static_0_1.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/test/test_static_0_2.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
test_static_cb = pl.read_csv(dataPath + "csv_files/test/test_static_cb_0.csv").pipe(set_table_dtypes)
test_person_1 = pl.read_csv(dataPath + "csv_files/test/test_person_1.csv").pipe(set_table_dtypes)
test_credit_bureau_b_2 = pl.read_csv(dataPath + "csv_files/test/test_credit_bureau_b_2.csv").pipe(set_table_dtypes)

## Join Tables & Feature Engineering

In [ ]:
# We need to use aggregation functions in tables with depth > 1, so tables that contain num_group1 column or also num_group2 column.
train_person_1_feats_1 = train_person_1.group_by("case_id").agg(
    pl.col("mainoccupationinc_384A").max().alias("mainoccupationinc_384A_max"),
    (pl.col("incometype_1044T") == "SELFEMPLOYED").max().alias("mainoccupationinc_384A_any_selfemployed")
)

# Here num_group1=0 has special meaning, it is the person who applied for the loan.
train_person_1_feats_2 = train_person_1.select(["case_id", "num_group1", "housetype_905L"]).filter(
    pl.col("num_group1") == 0
).drop("num_group1").rename({"housetype_905L": "person_housetype"})

# Here we have num_goup1 and num_group2, so we need to aggregate again.
train_credit_bureau_b_2_feats = train_credit_bureau_b_2.group_by("case_id").agg(
    pl.col("pmts_pmtsoverdue_635A").max().alias("pmts_pmtsoverdue_635A_max"),
    (pl.col("pmts_dpdvalue_108P") > 31).max().alias("pmts_dpdvalue_108P_over31")
)

selected_static_cols = []
for col in train_static.columns:
    if col[-1] in ("A", "M"):
        selected_static_cols.append(col)
print(selected_static_cols)

selected_static_cb_cols = []
for col in train_static_cb.columns:
    if col[-1] in ("A", "M"):
        selected_static_cb_cols.append(col)
print(selected_static_cb_cols)

# Join all tables together.
data = train_basetable.join(
    train_static.select(["case_id"]+selected_static_cols), how="left", on="case_id"
).join(
    train_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id"
).join(
    train_person_1_feats_1, how="left", on="case_id"
).join(
    train_person_1_feats_2, how="left", on="case_id"
).join(
    train_credit_bureau_b_2_feats, how="left", on="case_id"
)

['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'disbursedcredamount_1113A', 'downpmt_116A', 'inittransactionamount_650A', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastcancelreason_561M', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'maininc_215A', 'maxannuity_159A', 'maxannuity_4075009A', 'maxdebt4_972A', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'previouscontdistrict_112M', 'price_1097A', 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 'totinstallas

In [ ]:
test_person_1_feats_1 = test_person_1.group_by("case_id").agg(
    pl.col("mainoccupationinc_384A").max().alias("mainoccupationinc_384A_max"),
    (pl.col("incometype_1044T") == "SELFEMPLOYED").max().alias("mainoccupationinc_384A_any_selfemployed")
)

test_person_1_feats_2 = test_person_1.select(["case_id", "num_group1", "housetype_905L"]).filter(
    pl.col("num_group1") == 0
).drop("num_group1").rename({"housetype_905L": "person_housetype"})

test_credit_bureau_b_2_feats = test_credit_bureau_b_2.group_by("case_id").agg(
    pl.col("pmts_pmtsoverdue_635A").max().alias("pmts_pmtsoverdue_635A_max"),
    (pl.col("pmts_dpdvalue_108P") > 31).max().alias("pmts_dpdvalue_108P_over31")
)

data_submission = test_basetable.join(
    test_static.select(["case_id"]+selected_static_cols), how="left", on="case_id"
).join(
    test_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id"
).join(
    test_person_1_feats_1, how="left", on="case_id"
).join(
    test_person_1_feats_2, how="left", on="case_id"
).join(
    test_credit_bureau_b_2_feats, how="left", on="case_id"
)

In [ ]:
data = data.to_pandas()
data['date_decision'] = pd.to_datetime(data['date_decision'])
print(f"Date range: {data['date_decision'].min()} to {data['date_decision'].max()}")
print(f"Week range: {data['WEEK_NUM'].min()} to {data['WEEK_NUM'].max()}")

Date range: 2019-01-01 00:00:00 to 2020-10-05 00:00:00
Week range: 0 to 91


In [ ]:
# Pre-COVID training, COVID validation/testing
covid_start = pd.Timestamp('2020-03-01')
mid_covid = pd.Timestamp('2020-07-01')

# Split data based on these dates
train_data = data[data['date_decision'] < mid_covid]
test_data = data[data['date_decision'] >= mid_covid]

# Print split information
print(f"Training (pre-COVID): {len(train_data)} samples ({train_data['date_decision'].min()} to {train_data['date_decision'].max()})")
print(f"Test (later COVID): {len(test_data)} samples ({test_data['date_decision'].min()} to {test_data['date_decision'].max()})")

Training (pre-COVID): 1376419 samples (2019-01-01 00:00:00 to 2020-06-30 00:00:00)
Test (later COVID): 150240 samples (2020-07-01 00:00:00 to 2020-10-05 00:00:00)


In [ ]:
cols_pred = []
for col in data.columns:
    if col[-1].isupper() and col[:-1].islower():
        cols_pred.append(col)
print(cols_pred)

base_train = train_data[["case_id","date_decision", "WEEK_NUM","MONTH", "target"]]
y_train = train_data["target"]
X_train = train_data[cols_pred]


base_test = test_data[["case_id","date_decision", "WEEK_NUM","MONTH", "target"]]
y_test = test_data["target"]
X_test = test_data[cols_pred]

for df in [X_train, X_test]:
    df = convert_strings(df)

['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'disbursedcredamount_1113A', 'downpmt_116A', 'inittransactionamount_650A', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastcancelreason_561M', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'maininc_215A', 'maxannuity_159A', 'maxannuity_4075009A', 'maxdebt4_972A', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'previouscontdistrict_112M', 'price_1097A', 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 'totinstallas

/tmp/ipython-input-3344571743.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype("string").astype('category')
/tmp/ipython-input-3344571743.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(new_dtype)
/tmp/ipython-input-3344571743.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [ ]:
print(f"Train: {X_train.shape}")
print(f"Test: {X_test.shape}")

Train: (1376419, 48)
Test: (150240, 48)


In [ ]:
print(f"Train: {y_train.shape}")
print(f"Test: {y_test.shape}")

Train: (1376419,)
Test: (150240,)


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526659 entries, 0 to 1526658
Data columns (total 58 columns):
 #   Column                                   Non-Null Count    Dtype         
---  ------                                   --------------    -----         
 0   case_id                                  1526659 non-null  int64         
 1   date_decision                            1526659 non-null  datetime64[ns]
 2   MONTH                                    1526659 non-null  int64         
 3   WEEK_NUM                                 1526659 non-null  int64         
 4   target                                   1526659 non-null  int64         
 5   amtinstpaidbefduel24m_4187115A           965535 non-null   float64       
 6   annuity_780A                             1526659 non-null  float64       
 7   annuitynextmonth_57A                     1526655 non-null  float64       
 8   avginstallast24m_3658937A                901784 non-null   float64       
 9   avglnamtstart

## KS test

In [ ]:
numeric_features = data.select_dtypes(include=['int64', 'float64', 'float16', 'float32']).columns

In [ ]:
numeric_features=numeric_features.drop(["case_id", "MONTH","WEEK_NUM","target"])
numeric_features

Index(['amtinstpaidbefduel24m_4187115A', 'annuity_780A',
       'annuitynextmonth_57A', 'avginstallast24m_3658937A',
       'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A',
       'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A',
       'currdebtcredtyperange_828A', 'disbursedcredamount_1113A',
       'downpmt_116A', 'inittransactionamount_650A', 'lastapprcredamount_781A',
       'lastotherinc_902A', 'lastotherlnsexpense_631A',
       'lastrejectcredamount_222A', 'maininc_215A', 'maxannuity_159A',
       'maxannuity_4075009A', 'maxdebt4_972A', 'maxinstallast24m_3658928A',
       'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A',
       'maxpmtlast3m_4525190A', 'price_1097A', 'sumoutstandtotal_3546847A',
       'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A',
       'totinstallast1m_4525188A', 'pmtaverage_3A', 'pmtaverage_4527227A',
       'pmtaverage_4955615A', 'pmtssum_45A', 'mainoccupationinc_384A_max',
       'pmts_pmtsoverdue

In [ ]:
#two-sided: The null hypothesis is that the two distributions are identical, F(x)=G(x) for all x;
#the alternative is that they are not identical. The statistic is the maximum absolute difference
#between the empirical distribution functions of the samples.
reference_data = train_data
current_data = test_data

for column in numeric_features:
    ks_stat, p_value = stats.ks_2samp(reference_data[column], current_data[column])
    if p_value < 0.05:
        print(f"Drift detected in {column}: p-value = {p_value:.4f}")

Drift detected in annuity_780A: p-value = 0.0000
Drift detected in credamount_770A: p-value = 0.0000
Drift detected in disbursedcredamount_1113A: p-value = 0.0000
Drift detected in downpmt_116A: p-value = 0.0000
Drift detected in mainoccupationinc_384A_max: p-value = 0.0000


## PSI - Numeric Features

In [ ]:
def psi(reference, monitored, feature, bins=None):


    reference_valid = reference[feature].dropna()
    monitored_valid = monitored[feature].dropna()

    if len(reference_valid) < 10 or len(monitored_valid) < 10:
      print(f"Warning: Feature '{feature}' has too few valid values. Skipping.")
    else:
      full_dataset = np.concatenate((reference_valid, monitored_valid))
      if bins is None:
          _, bin_edges = np.histogram(full_dataset, bins="doane")
      else:
          bin_edges = np.linspace(min(min(reference_valid), min(monitored_valid)), max(max(reference_valid), max(monitored_valid)), bins + 1)

      # Calculate the histogram for each dataset
      reference_hist, _ = np.histogram(reference_valid, bins=bin_edges)
      monitored_hist, _ = np.histogram(monitored_valid, bins=bin_edges)

      # Convert histograms to proportions
      reference_proportions = reference_hist / np.sum(reference_hist)
      monitored_proportions = monitored_hist / np.sum(monitored_hist)

      # Replace zeroes to avoid division by zero or log of zero errors
      monitored_proportions = np.where(monitored_proportions == 0, 1e-6, monitored_proportions)
      reference_proportions = np.where(reference_proportions == 0, 1e-6, reference_proportions)

      # Calculate PSI
      psi_values = (monitored_proportions - reference_proportions) * np.log(monitored_proportions / reference_proportions)
      psi = np.sum(psi_values)

      return psi

In [ ]:
results = []

for i in numeric_features:
    psi_value = psi(train_data, test_data, i, bins=10)

    if psi_value == None:
        continue
    elif psi_value < 0.1:
        shift = 'No shift'
    elif psi_value < 0.25:
        shift = 'Moderate shift'
    else:
        shift = 'Significant shift'

    results.append({'Feature': i, 'PSI': psi_value, 'Shift': shift})


psi_df = pd.DataFrame(results)


In [ ]:
psi_df[psi_df['Shift'] == 'Significant shift']

,Feature,PSI,Shift


In [ ]:
psi_df[psi_df['Shift'] == 'Moderate shift']

,Feature,PSI,Shift
14,lastotherinc_902A,0.172801,Moderate shift


## PSI - Categorical features

In [ ]:
categorical_features = data.select_dtypes(include=['object', 'category']).columns
categorical_features

Index(['lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M',
       'lastcancelreason_561M', 'lastrejectcommoditycat_161M',
       'lastrejectcommodtypec_5251769M', 'lastrejectreason_759M',
       'lastrejectreasonclient_4145040M', 'previouscontdistrict_112M',
       'description_5085714M', 'education_1103M', 'education_88M',
       'maritalst_385M', 'maritalst_893M', 'person_housetype',
       'pmts_dpdvalue_108P_over31'],
      dtype='object')

In [ ]:
def calculate_psi_categorical(expected_df, actual_df, feature, min_samples=10):

    # Convert to Series and drop NaNs
    expected = pd.Series(expected_df[feature]).dropna()
    actual = pd.Series(actual_df[feature]).dropna()

    # Check for minimum sample size
    if len(expected) < min_samples or len(actual) < min_samples:
        print(f"Not enough data to calculate PSI. Require at least {min_samples} non-NaN values in each dataset.")
        return None

    # Get normalized value counts (proportions)
    expected_dist = expected.value_counts(normalize=True)
    actual_dist = actual.value_counts(normalize=True)

    # All categories from both sets
    all_categories = set(expected_dist.index).union(set(actual_dist.index))
    floor = 1e-6
    psi = 0

    for cat in all_categories:
        expected_pct = max(expected_dist.get(cat, 0.0), floor)
        actual_pct = max(actual_dist.get(cat, 0.0), floor)

        psi += (expected_pct - actual_pct) * np.log(expected_pct / actual_pct)

    return psi


In [ ]:
cat_results = []

for i in categorical_features:
    psi_value = calculate_psi_categorical(train_data, test_data, i)

    if psi_value == None:
        continue
    elif psi_value < 0.1:
        shift = 'No shift'
    elif psi_value < 0.25:
        shift = 'Moderate shift'
    else:
        shift = 'Significant shift'

    cat_results.append({'Feature': i, 'PSI': psi_value, 'Shift': shift})


psi_df_cat = pd.DataFrame(cat_results)


In [ ]:
psi_df_cat[psi_df_cat['Shift'] == 'Significant shift']

,Feature,PSI,Shift
1,lastapprcommoditytypec_5251766M,1.374343,Significant shift
4,lastrejectcommodtypec_5251769M,0.699762,Significant shift
8,description_5085714M,6.394625,Significant shift


In [ ]:
psi_df_cat[psi_df_cat['Shift'] == 'Moderate shift']

,Feature,PSI,Shift
0,lastapprcommoditycat_1041M,0.139418,Moderate shift
7,previouscontdistrict_112M,0.132431,Moderate shift


In [ ]:
df=pd.concat([psi_df[psi_df['Shift'] == 'Moderate shift'],
              psi_df_cat[psi_df_cat['Shift'] == 'Moderate shift'],
              psi_df[psi_df['Shift'] == 'Significant shift'],
              psi_df_cat[psi_df_cat['Shift'] == 'Significant shift']], ignore_index = True )

In [ ]:
df

,Feature,PSI,Shift
0,lastotherinc_902A,0.172801,Moderate shift
1,lastapprcommoditycat_1041M,0.139418,Moderate shift
2,previouscontdistrict_112M,0.132431,Moderate shift
3,lastapprcommoditytypec_5251766M,1.374343,Significant shift
4,lastrejectcommodtypec_5251769M,0.699762,Significant shift
5,description_5085714M,6.394625,Significant shift


## Two Proportion z-test

In [ ]:
from statsmodels.stats.proportion import proportions_ztest


ref_positive = sum(y_train)
ref_total = len(y_train)

curr_positive = sum(y_test)
curr_total = len(y_test)

# Prepare data for test
counts = [ref_positive, curr_positive]
nobs = [ref_total, curr_total]

# Perform two-proportion z-test
z_stat, p_value = proportions_ztest(counts, nobs)

# Results
ref_rate = ref_positive / ref_total
curr_rate = curr_positive / curr_total

print(f"Reference P(y=1): {ref_rate:.3f}")
print(f"Current P(y=1): {curr_rate:.3f}")
print(f"Z-statistic: {z_stat:.3f}")
print(f"P-value: {p_value:.3f}")


alpha = 0.05
if p_value < alpha:
    print(f"Drift detected! (p < {alpha})")
else:
    print(f"No significant drift (p >= {alpha})")

Reference P(y=1): 0.033
Current P(y=1): 0.021
Z-statistic: 24.106
P-value: 0.000
Drift detected! (p < 0.05)
